# Volumetric maging speed for a given sample size
- sample volume of 1 cm3
- camera Photometric Iris 15, FOV = 21.49 x 12.61 mm
- distance between z-planes 5 um (undersampling, but preserving cellular resolution)

In [1]:
import numpy as np

In [52]:
MAG = 2 # detection objective magnification
frames_per_s = 1000/267.
FOV_Y_mm, FOV_X_mm = 21.49, 12.61
FOV_Y_px, FOV_X_px = 5056, 2960
bytes_per_px = 2 # 16-bit camera
FOV_X_sample, FOV_Y_sample = FOV_X_mm/MAG, FOV_Y_mm/MAG
z_step = 0.005
sample_dim_z_mm = 10
sample_dim_x_mm = sample_dim_y_mm = 10
tile_overlap = 0.1 # 10%

In [55]:
# N_tiles are allowed to be non-integer, because this is averaged approximation
N_planes = sample_dim_z_mm/z_step
N_tiles_X, N_tiles_Y = sample_dim_x_mm / (FOV_X_sample * (1 - tile_overlap)), sample_dim_y_mm / (FOV_Y_sample * (1 - tile_overlap))
print(f"N_tiles_X, N_tiles_Y = {N_tiles_X:.1f}, {N_tiles_Y:.1f} (can be non-integer, approximation for larger volumes)")

N_tiles_X, N_tiles_Y = 1.8, 1.0 (can be non-integer, approximation for larger volumes)


In [51]:
N_frames = int(N_planes * N_tiles_X * N_tiles_Y)
total_time_s = int(N_frames / frames_per_s)
total_GB = N_frames * FOV_Y_px * FOV_X_px * bytes_per_px / 1e9
print(f"Total imaging time per 1 cm3 sample volume: {total_time_s} seconds")
print(f"Dataset size (single channel): {int(total_GB)} GB")

Total imaging time per 1 cm3 sample volume: 972 seconds
Dataset size (single channel): 109 GB
